# 1. load and split image set

In [1]:
from torchvision.datasets import ImageFolder              # for loading images from ImageNet
from torch.utils.data import DataLoader, random_split

def split_data_set(dataset, random_seed):
    if random_seed is not None:
        torch.manual_seed(random_seed)  # set input as random seed

    # split image set ---> half for calibration data set, half for test data set
    dataset_length = len(dataset)
    print(f"Samples amount: {dataset_length}")
    calib_length = dataset_length // 2               
    test_length = dataset_length - calib_length      

    calib_dataset, test_dataset = random_split(dataset, [calib_length, test_length])
    return calib_dataset, test_dataset

# 2. Calculate Conformal Score

In [2]:
import numpy as np

# conformal function s(x,y)
def conformal_scores(model, dataloader, alpha=0.1, lambda_reg=0.1, k_reg=5):
    scores = []  # conformal scores of image sets
    labels = []  # true label sets 
    with torch.no_grad():
        for images, true_labels in dataloader:
            images, true_labels = images.to(device), true_labels.to(device)
            # logistic value
            outputs = model(images)
            # logistic value -> softmax
            # dim=1 : convert logistic values for all the classes of the example to the softmax 
            softmaxs = torch.softmax(outputs, dim=1)
            
            for softmax, true_label in zip(softmaxs, true_labels):
                # descending sort softmax
                sorted_softmax, sorted_index = torch.sort(softmax, descending=True)
                
                # get the position of the true label in the sorted softmax
                true_label_position = (sorted_index == true_label).nonzero(as_tuple=True)[0].item()
                # independent random variable u ~ Uniform(0, 1)
                u = np.random.uniform(0, 1)
                # cumulate sorted softmax
                cumulative_softmax = torch.cumsum(sorted_softmax, dim=0)  # dim=0 -> cumulate by raw direction

                # calculate p_x(y) + pi_x(y)*u
                if true_label_position == 0:
                    # first softmax is true label -> p_x(y) = 0 ; pi_x(y) = sorted_softmax[0]
                    p_and_pi = u * sorted_softmax[true_label_position].item()
                else:
                    p_and_pi = cumulative_softmax[true_label_position - 1].item() + u * sorted_softmax[true_label_position].item()

                # calculate regularization term: lamba * ( o_x(y) - k_reg)+
                regularization_term = lambda_reg * max( true_label_position + 1 - k_reg, 0)
                conformal_score = p_and_pi + regularization_term
                    
                scores.append(conformal_score)
                labels.append(true_label.item())
    return np.array(scores), np.array(labels)

## 3. Construct APS

In [3]:
def raps_classification(model, dataloader, t_cal, lambda_reg=0.1, k_reg=5):
    raps = []         # probability set
    raps_labels = []  # label set indicated to the probability set
    labels = []       # true label
    with torch.no_grad():
        for images, true_labels in dataloader:
            images, true_labels = images.to(device), true_labels.to(device)
            outputs = model(images)
            softmaxs = torch.softmax(outputs, dim=1)

            # sort softmax probabilities
            sorted_softmax, sorted_indices = torch.sort(softmaxs, descending=True, dim=1)  # shape: [batch_size, 1000]
            cumulative_softmax = torch.cumsum(sorted_softmax, dim=1)                       # shape: [batch_size, 1000]

            # rank of current sorted probability: [1,2,3,...,1000]
            rank = torch.arange(1, sorted_softmax.size(1) + 1, device=device).unsqueeze(0)  # shape: [1, 1000]
            # calculate regularization term
            regularization_term = lambda_reg * torch.clamp(rank - k_reg, min=0)             # shape: [1, 1000]

            # generate random variable u for all samples
            u = torch.rand_like(sorted_softmax)  # shape: [batch_size, 1000]

            # E = cumulative[current-1] + u*sorted[current] + regularization
            # which is equal to: cumulative[current] - sorted[current] + u*sorted[current] + regularization
            e = cumulative_softmax - sorted_softmax + u * sorted_softmax + regularization_term  # shape: [batch_size, 1000]

            e_less_than_t = e <= t_cal  
            # the first index of e that exceeds t_cal
            # if all the e <= t_cal  -> return 0
            e_exceed_t = (e > t_cal).float().argmax(dim=1) 
            cutoff_indices = torch.maximum(torch.sum(e_less_than_t, dim=1), e_exceed_t)  

            # build prediction sets
            for i, cutoff_index in enumerate(cutoff_indices):
                cutoff_index = max(cutoff_index.item(), 1)  # ensure cutoff_index >= 1
                raps.append(sorted_softmax[i, :cutoff_index].tolist())  
                raps_labels.append(sorted_indices[i, :cutoff_index].tolist())  
                labels.append(true_labels[i].item())  
    return raps, raps_labels, labels

## 4. Evaluate Perdiction Set

In [4]:
def eval_aps(aps_labels,  true_labels):
    total_set_size = 0
    coveraged = 0
    for aps_label, true_label in zip(aps_labels, true_labels):
        # cumulate total set size
        total_set_size += len(aps_label)
        # cumulate the predictions sets if it contains true label
        if true_label in aps_label:
            coveraged += 1

    # calculate average values
    samples_amount = len(true_labels)
    average_set_size = total_set_size / samples_amount
    average_coverage = coveraged / samples_amount
    print(f"Total set size: {total_set_size}")
    print(f"Total coverage sets: {coveraged}")
    print(f"Total samples amount: {samples_amount}")
    return average_set_size, average_coverage

## 5. Construct APS repeatedly

In [5]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR100        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import timm
import detectors

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet50_cifar100", pretrained=True)
model.to(device)
model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1
lambda_reg = 0.1
k_reg = 3

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="./data", train=False, download=True, transform=data_transform)

# contruct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("RAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = conformal_scores(model, calib_loader, alpha, lambda_reg, k_reg)
    t_cal = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, t_cal, lambda_reg, k_reg)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Ratein runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)

print(f"Final Average Prediction Set Size: {final_avg_set_size}")
print(f"Final Average Coverage: {final_avg_coverage}")

C:\Users\jiayang\anaconda3\envs\pytorch2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
RAPS Classification, Start!

Running experiment 1/10...
Samples amount: 10000
t_cal = 0.9858953403941191
Total set size: 8871
Total coverage sets: 4560
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 1.7742
Average Coverage Ratein runs 1: 0.912

Running experiment 2/10...
Samples amount: 10000
t_cal = 0.9856299096759306
Total set size: 9001
Total coverage sets: 4557
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 1.8002
Average Coverage Ratein runs 2: 0.9114

Running experiment 3/10...
Samples amount: 10000
t_cal = 0.9842405295429275
Total set size: 8875
Total coverage sets: 4547
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 1.775
Average Coverage Ratein runs 3: 0.9094

Running experiment 4/10...
Samples amount: 10000
t_cal = 0.9828839223029777
Total set size: 8740
Total coverage sets:

# Result

- Final Average **Prediction Set Size： 1.77 / 100**
- Final Average **Coverage: 90.82% ($\alpha$=0.1)**